In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('/home/tammy.kojima/Authorship-attribution/')
import feature_extraction as fe
from tqdm import tqdm
import pickle
import os
import ast
from sklearn.model_selection import train_test_split

/home/tammy.kojima/miniconda3/envs/meu_ambiente/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
df_mistral = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_mistral.csv')
df_gpt = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt.csv')
df_human = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_human.csv')
df_mistral = df_mistral.iloc[:5000]
df_gpt = df_gpt.iloc[:5000]
df_human = df_human.iloc[:10000]
df_test = pd.concat([df_gpt, df_mistral, df_human], ignore_index=True)
df_test

,text,origin,model,tweet_id,topic,length,batch_timestamp,characters_remaining
0,"Our rules are the most robust, the president c...",AI,chat gpt,NaN,NaN,NaN,NaN,NaN
1,...associating outsiders with danger and extre...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN
2,...alarms were raised in the initial stages of...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN
4,[The governor's] fixation with health protocol...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
19995,@KraziiKayy Speaking of Anna we got a 95 on th...,human,NaN,NaN,NaN,NaN,NaN,NaN
19996,@garyshort I somehow feel you were at a distin...,human,NaN,NaN,NaN,NaN,NaN,NaN
19997,"@joshtastic1 Oh I dunno,,I couldnt explain thi...",human,NaN,NaN,NaN,NaN,NaN,NaN
19998,Yeah T just go to http://www.youravon.com/sall...,human,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Tokeniza todos os textos primeiro
textos = df_test["text"].fillna("").tolist()
processed_texts = [fe.preprocess_text(texto) for texto in tqdm(textos, desc="Tokenizando textos")]

Tokenizando textos:   0%|                                                                                                                        | 0/20000 [00:00<?, ?it/s][nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
Tokenizando textos: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:04<00:00, 4104.88it/s]


In [ ]:
df_prossessed_texts = pd.DataFrame({'text': processed_texts})
df_prossessed_texts.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_geral.csv', index=False)

In [2]:
df_prossessed_texts = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_geral.csv')
df_prossessed_texts

,text
0,"['our', 'rules', 'are', 'the', 'most', 'robust..."
1,"['...', 'associating', 'outsiders', 'with', 'd..."
2,"['...', 'alarms', 'were', 'raised', 'in', 'the..."
3,"['[', 'the', 'opposition', ']', 'utilizes', 'e..."
4,"['[', 'the', ""governor's"", ']', 'fixation', 'w..."
...,...
19995,"['@kraziikayy', 'speaking', 'of', 'anna', 'we'..."
19996,"['@garyshort', 'i', 'somehow', 'feel', 'you', ..."
19997,"['@joshtastic1', 'oh', 'i', 'dunno', ',', ',',..."
19998,"['yeah', 't', 'just', 'go', 'to', 'http://www...."


In [8]:
# Lista para armazenar resultados
lexical_features = []

for texto in tqdm(textos):
    # Tokeniza o texto usando Twokenizer
    words = fe.preprocess_text(texto)  # já retorna tokens em lowercase
    
    # Extrai features léxicas
    features = fe.extract_lexical_features(texto, words)
    lexical_features.append(features)

df_lexical = pd.DataFrame(lexical_features)
df_test = pd.concat([df_test, df_lexical], axis=1)
df_test.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 18275.58it/s]


,text,origin,model,tweet_id,topic,length,batch_timestamp,characters_remaining,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.818182,22,18,4.318182,8.216942,0.181818
1,...associating outsiders with danger and extre...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.952381,21,20,5.476190,11.392290,0.190476
2,...alarms were raised in the initial stages of...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.914286,35,32,4.714286,5.632653,0.228571
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.760000,25,19,4.880000,9.785600,0.360000
4,[The governor's] fixation with health protocol...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.933333,15,14,5.533333,10.782222,0.200000


In [9]:
stylistic_features = [
    fe.extract_stylistic_features(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_stylistic = pd.DataFrame(stylistic_features)
df_test = pd.concat([df_test, df_stylistic], axis=1)
df_test.head()

Extraindo features estilísticas: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 125719.68it/s]


,text,origin,model,tweet_id,topic,length,batch_timestamp,characters_remaining,lexical_type_token_ratio,lexical_word_count,...,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density
0,"Our rules are the most robust, the president c...",AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.818182,22,...,4.318182,8.216942,0.181818,0,0,0.0,0.0,0,0.0,0.0
1,...associating outsiders with danger and extre...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.952381,21,...,5.476190,11.392290,0.190476,1,0,0.0,0.0,1,0.0,0.0
2,...alarms were raised in the initial stages of...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.914286,35,...,4.714286,5.632653,0.228571,1,0,0.0,0.0,1,0.0,0.0
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.760000,25,...,4.880000,9.785600,0.360000,0,0,0.0,0.0,0,0.0,0.0
4,[The governor's] fixation with health protocol...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.933333,15,...,5.533333,10.782222,0.200000,0,0,0.0,0.0,0,0.0,0.0


In [10]:
structural_features = [
    fe.extract_structural_features(textos[i], processed_texts[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estruturais")
]
df_structural = pd.DataFrame(structural_features)
df_test = pd.concat([df_test, df_structural], axis=1)
df_test.head()

Extraindo features estruturais: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 155212.09it/s]


,text,origin,model,tweet_id,topic,length,batch_timestamp,characters_remaining,lexical_type_token_ratio,lexical_word_count,...,stylistic_emoticon_density,structural_has_url,structural_has_mention,structural_has_hashtag,structural_is_retweet,structural_url_density,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers
0,"Our rules are the most robust, the president c...",AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.818182,22,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,...associating outsiders with danger and extre...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.952381,21,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,...alarms were raised in the initial stages of...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.914286,35,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.760000,25,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,[The governor's] fixation with health protocol...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.933333,15,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [11]:
syntactic_features = [
    fe.extract_syntactic_features(tokens)
    for tokens in tqdm(processed_texts, desc="Extraindo features sintáticas")
]
df_syntactic = pd.DataFrame(syntactic_features)
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

Extraindo features sintáticas: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [01:11<00:00, 281.50it/s]


,text,origin,model,tweet_id,topic,length,batch_timestamp,characters_remaining,lexical_type_token_ratio,lexical_word_count,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.818182,22,...,0.0,0.0,0.0,0.0,2.032114,2.780466,17.0,0.0,0.090909,0.227273
1,...associating outsiders with danger and extre...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.952381,21,...,0.0,0.0,0.0,0.0,1.971333,2.902002,9.0,0.0,0.047619,0.238095
2,...alarms were raised in the initial stages of...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.914286,35,...,0.0,0.0,0.0,0.0,2.089713,3.198872,32.0,0.0,0.000000,0.114286
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.760000,25,...,0.0,0.0,0.0,0.0,1.817905,2.375897,22.0,0.0,0.000000,0.120000
4,[The governor's] fixation with health protocol...,AI,chat gpt,NaN,NaN,NaN,NaN,NaN,0.933333,15,...,0.0,0.0,0.0,0.0,1.808046,2.523211,13.0,0.0,0.000000,0.200000


In [12]:
ngrams_df = fe.extract_ngrams_features(processed_texts, max_features=300)
ngrams_df.head()

,about,adaptive,after,again,ai,ai can,ai in,ai is,ai is revolutionizing,ai powered,...,will,with,with ai,work,world,would,yes,you,you know,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [13]:
# Resetar os índices de ambos os DataFrames
df_test_reset = df_test.reset_index(drop=True)
ngrams_df_reset = ngrams_df.reset_index(drop=True)

# Agora concatenar
df_combined = pd.concat([df_test_reset, ngrams_df_reset], axis=1)
df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})
df_combined.fillna(0)
df_combined.head()

/tmp/ipykernel_1093876/3577768971.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})


,text,origin,model,tweet_id,topic,length,batch_timestamp,characters_remaining,lexical_type_token_ratio,lexical_word_count,...,will,with,with ai,work,world,would,yes,you,you know,your
0,"Our rules are the most robust, the president c...",1.0,chat gpt,NaN,NaN,NaN,NaN,NaN,0.818182,22,...,0,0,0,0,0,0,0,0,0,0
1,...associating outsiders with danger and extre...,1.0,chat gpt,NaN,NaN,NaN,NaN,NaN,0.952381,21,...,0,1,0,0,0,0,0,0,0,0
2,...alarms were raised in the initial stages of...,1.0,chat gpt,NaN,NaN,NaN,NaN,NaN,0.914286,35,...,0,0,0,0,0,0,0,0,0,0
3,[The opposition] utilizes every tool to questi...,1.0,chat gpt,NaN,NaN,NaN,NaN,NaN,0.760000,25,...,0,0,0,0,0,0,0,0,0,0
4,[The governor's] fixation with health protocol...,1.0,chat gpt,NaN,NaN,NaN,NaN,NaN,0.933333,15,...,0,1,0,0,0,0,0,0,0,0


In [14]:
df_combined = df_combined.drop(columns=['topic', 'model', 'tweet_id', 'text', 'length', 'batch_timestamp', 'characters_remaining']) 
df_combined.head()

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,will,with,with ai,work,world,would,yes,you,you know,your
0,1.0,0.818182,22,18,4.318182,8.216942,0.181818,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.952381,21,20,5.476190,11.392290,0.190476,1,0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,1.0,0.914286,35,32,4.714286,5.632653,0.228571,1,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.760000,25,19,4.880000,9.785600,0.360000,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0.933333,15,14,5.533333,10.782222,0.200000,0,0,0.0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
df_combined['origin'] = df_combined['origin'].fillna(1)
df_combined.head()

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,will,with,with ai,work,world,would,yes,you,you know,your
0,1.0,0.818182,22,18,4.318182,8.216942,0.181818,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.952381,21,20,5.476190,11.392290,0.190476,1,0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,1.0,0.914286,35,32,4.714286,5.632653,0.228571,1,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.760000,25,19,4.880000,9.785600,0.360000,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0.933333,15,14,5.533333,10.782222,0.200000,0,0,0.0,...,0,1,0,0,0,0,0,0,0,0


In [17]:
df_combined.to_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_geral_com_features.csv", index=False)

In [4]:
df_combined = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_geral_com_features2.csv')
df_combined

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,chatgpt_structured_output,chatgpt_overly_polite,chatgpt_disclaimer_density,chatgpt_assistant_patterns,mistral_self_ref,mistral_structured_density,mistral_technical_jargon,mistral_non_english_density,mistral_step_reasoning,mistral_low_ethical_disclaimers
0,1.0,0.818182,22,18,4.318182,8.216942,0.181818,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.952381,21,20,5.476190,11.392290,0.190476,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.914286,35,32,4.714286,5.632653,0.228571,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.760000,25,19,4.880000,9.785600,0.360000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.933333,15,14,5.533333,10.782222,0.200000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.758621,29,22,3.689655,5.317479,0.344828,0,0,0.103448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.0,1.000000,17,17,4.352941,10.581315,0.352941,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,0.0,0.925926,27,25,3.925926,5.994513,0.185185,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,0.0,0.818182,22,18,4.727273,46.834711,0.181818,1,0,0.090909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_combined = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_geral_com_features.csv')
X_new_features = fe.extract_features_batch(textos, processed_texts = processed_texts)
df_features = pd.DataFrame(X_new_features)
df_combined = pd.concat([df_combined, df_features], axis=1)
df_combined.head()

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,chatgpt_structured_output,chatgpt_overly_polite,chatgpt_disclaimer_density,chatgpt_assistant_patterns,mistral_self_ref,mistral_structured_density,mistral_technical_jargon,mistral_non_english_density,mistral_step_reasoning,mistral_low_ethical_disclaimers
0,1.0,0.818182,22,18,4.318182,8.216942,0.181818,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.952381,21,20,5.476190,11.392290,0.190476,1,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.914286,35,32,4.714286,5.632653,0.228571,1,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.760000,25,19,4.880000,9.785600,0.360000,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.933333,15,14,5.533333,10.782222,0.200000,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_combined.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_geral_com_features2.csv', index=False)